# Adjacency Matrix (AM)

>hash-addressable, dynamically-growing, PyTorch-only implementation

```math
r(hᵢ, hⱼ) ← v  (\text{ overwrite if exists, insert if new})
```

for an arbitrary number of tokens whose only identifier is the hash value h = hash(tᵢ).


No Python loops touch the tensor itself; all lookups and updates are O(1) and executed with one tensor op.

## Data layout

We keep a single 2-D tensor R of shape (capacity, capacity)
row/col indices are internal integer ids (0,1,2,…) that we map to/from the hash values with two dictionaries:

```text
hash → idx   : self.h2i
idx  → hash  : self.i2h
```

capacity grows automatically in powers of two (doubling when >75 % full) and the old contents are copied into the new larger tensor with a single index_copy.

In [ ]:
import torch
import os

# Check available GPUs
print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    print(f"  GPU {i}: {props.name}")
    print(f"    Compute Capability: sm_{props.major}{props.minor}")
    print(f"    Memory: {props.total_memory / 1024**3:.2f} GB")

# Select RTX 3060 (adjust index based on output above)
# Option A: Hide Quadro, only show RTX
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Assuming RTX is at index 1
DEVICE = torch.device("cuda:0")  # Now index 0 refers to the RTX

# Option B: Explicitly select by index (if you don't set CUDA_VISIBLE_DEVICES)
# DEVICE = torch.device("cuda:1")  # Direct access to RTX at original index 1

print(f"\nUsing device: {DEVICE}")
if DEVICE.type == "cuda":
    print(f"Device name: {torch.cuda.get_device_name(DEVICE)}")
    print(f"Memory: {torch.cuda.get_device_properties(DEVICE).total_memory / 1024**3:.2f} GB")

In [12]:
import torch
from typing import List, Dict, Optional

class HashRelationTensor:
    """
    Binary relation tensor r(h_i, h_j) = v  addressed only by hashes.
    Shape grows automatically; all updates are one tensor op.
    """
    def __init__(self, init_cap=16, dtype=torch.float32, device='cpu'):
        self.device   = device
        self.dtype    = dtype
        self.capacity = init_cap
        self.R        = torch.zeros(init_cap, init_cap, dtype=dtype, device=device)

        self.h2i      = {}          # hash -> internal index
        self.i2h      = {}          # internal index -> hash
        self.next_idx = 0           # first free internal index

    # ---------- public ----------
    def update(self, h_i: int, h_j: int, value):
        """h_i, h_j are hash values (python ints)"""
        # obtain (or create) internal indices
        i = self._index_for_hash(h_i)
        j = self._index_for_hash(h_j)
        # single in-place tensor write
        self.R[i, j] = value

    def get(self, h_i: int, h_j: int):
        if h_i not in self.h2i or h_j not in self.h2i:
            raise KeyError("Relation not stored")
        return self.R[self.h2i[h_i], self.h2i[h_j]]

    def get_dense(self) -> torch.Tensor:
        """Return the dense sub-matrix actually in use (shape next_idx × next_idx)"""
        return self.R[:self.next_idx, :self.next_idx].clone()
    
    def prune(self, keep: List[int]) -> torch.Tensor:
        """
        Drop every token whose hash is NOT in `keep`.
        Rebuild h2i/i2h and shrink self.R to |keep|×|keep| in-place.
        Returns the dense pruned adjacency matrix.
        """
        keep_set = set(keep)
        # 1. build new contiguous index mapping
        new_h2i = {h: idx for idx, h in enumerate(h for h in keep if h in self.h2i)}
        n = len(new_h2i)
        if n == 0:                       # nothing left
            self.h2i.clear()
            self.i2h.clear()
            self.next_idx = 0
            self.capacity = 0
            self.R = torch.empty(0, 0, dtype=self.dtype, device=self.device)
            return self.R

        # 2. allocate compact matrix
        new_R = torch.zeros(n, n, dtype=self.dtype, device=self.device)

        # 3. copy old values into new positions (vectorised)
        old_rows, old_cols = [], []
        new_rows, new_cols = [], []
        vals = []
        for h_row in new_h2i:
            for h_col in new_h2i:
                old_rows.append(self.h2i[h_row])
                old_cols.append(self.h2i[h_col])
                new_rows.append(new_h2i[h_row])   # ✅ use NEW indices
                new_cols.append(new_h2i[h_col])
                vals.append(self.R[self.h2i[h_row], self.h2i[h_col]])

        if vals:
            new_R[torch.tensor(new_rows, device=self.device),
                torch.tensor(new_cols, device=self.device)] = torch.tensor(vals, device=self.device)

        # 4. overwrite internal state
        self.h2i = new_h2i
        self.i2h = {idx: h for h, idx in new_h2i.items()}
        self.R = new_R
        self.capacity = n
        self.next_idx = n
        return new_R
    
    # ====== history (incoming edges) ======
    def get_history(self, h: int, horizon: int = 1) -> Dict[int, float]:
        """Return {hash_of_past_token : edge_strength} within horizon steps."""
        if h not in self.h2i:
            return {}
        idx = self.h2i[h]
        # 1-step predecessors
        mask = self.R[:, idx] != 0          # (capacity,)
        preds = mask.nonzero(as_tuple=False).squeeze(1)  # (<=capacity,)
        strengths = self.R[preds, idx]      # (<=capacity,)
        # truncate to horizon closest (here 1 step – see multi-step below)
        topk = min(horizon, len(preds))
        if topk == 0:
            return {}
        top_vals, top_idx = torch.topk(strengths, k=topk)
        return {self.i2h[int(i)]: float(v) for i, v in zip(preds[top_idx], top_vals)}

    def get_history_batch(self, hs: List[int], horizon: int = 1) -> List[Dict[int, float]]:
        return [self.get_history(h, horizon) for h in hs]

    # ====== future (outgoing edges) ======
    def get_future(self, h: int, horizon: int = 1) -> Dict[int, float]:
        """Return {hash_of_future_token : edge_strength} within horizon steps."""
        if h not in self.h2i:
            return {}
        idx = self.h2i[h]
        mask = self.R[idx, :] != 0
        succs = mask.nonzero(as_tuple=False).squeeze(1)
        strengths = self.R[idx, succs]
        topk = min(horizon, len(succs))
        if topk == 0:
            return {}
        top_vals, top_idx = torch.topk(strengths, k=topk)
        return {self.i2h[int(i)]: float(v) for i, v in zip(succs[top_idx], top_vals)}

    def get_future_batch(self, hs: List[int], horizon: int = 1) -> List[Dict[int, float]]:
        return [self.get_future(h, horizon) for h in hs]

    # ====== projection (dense sub-matrix for a given token set) ======
    def project(self, hs: List[int]) -> torch.Tensor:
        """Return dense |hs|×|hs| relation matrix for the supplied hashes."""
        idx = [self.h2i[h] for h in hs if h in self.h2i]
        if not idx:
            return torch.empty(0, 0, dtype=self.dtype, device=self.device)
        idx_t = torch.tensor(idx, device=self.device)
        return self.R[idx_t][:, idx_t]   # elegant batched indexing

    # ---------- internal ----------
    def _index_for_hash(self, h):
        """Return internal index for hash h, growing tensor if necessary."""
        if h in self.h2i:
            return self.h2i[h]
        # new hash → assign next internal index
        if self.next_idx >= self.capacity:
            self._grow()
        idx = self.next_idx
        self.h2i[h] = idx
        self.i2h[idx] = h
        self.next_idx += 1
        return idx

    def _grow(self):
        old_cap = self.capacity
        new_cap = old_cap * 2
        new_R = torch.zeros(new_cap, new_cap, dtype=self.dtype, device=self.device)
        # copy old block in one tensor op
        new_R[:old_cap, :old_cap] = self.R
        self.R = new_R
        self.capacity = new_cap

## Usage demo

In [13]:
dev = 'cuda' if torch.cuda.is_available() else 'cpu'
rel = HashRelationTensor(device=dev)

tok_hashes = [hash(t) for t in ["cat", "dog", "bird"]]

rel.update(tok_hashes[0], tok_hashes[1], 0.8)   # r(cat,dog)=0.8
rel.update(tok_hashes[1], tok_hashes[2], 0.5)   # r(dog,bird)=0.5
rel.update(tok_hashes[0], tok_hashes[1], 0.9)   # overwrite with 0.9

print(rel.get(tok_hashes[0], tok_hashes[1]))    # tensor(0.9, device='cuda:0')
print(rel.get_dense())                          # 3×3 dense sub-matrix

tensor(0.9000, device='cuda:0')
tensor([[0.0000, 0.9000, 0.0000],
        [0.0000, 0.0000, 0.5000],
        [0.0000, 0.0000, 0.0000]], device='cuda:0')


In [14]:
rel = HashRelationTensor()

# build a tiny time graph
t = [hash(w) for w in ["t0", "t1", "t2", "t3"]]
rel.update(t[0], t[1], 0.9)   # t0 → t1
rel.update(t[1], t[2], 0.8)   # t1 → t2
rel.update(t[2], t[3], 0.7)   # t2 → t3
rel.update(t[0], t[3], 0.5)   # t0 → t3  (skip connection)

print(rel.get_history(t[3], horizon=2))
# {hash(t2): 0.7, hash(t0): 0.5}   two strongest predecessors

print(rel.get_future(t[0], horizon=2))
# {hash(t1): 0.9, hash(t3): 0.5}   two strongest successors

print(rel.project([t[0], t[2], t[3]]))
# 3×3 dense tensor of relations among exactly these three tokens

{2053597437022370697: 0.699999988079071, 895199035842044268: 0.5}
{5354350517194846248: 0.8999999761581421, -732056921184991155: 0.5}
tensor([[0.0000, 0.0000, 0.5000],
        [0.0000, 0.0000, 0.7000],
        [0.0000, 0.0000, 0.0000]])


## Pruning Demo

### Complexity

Time: O(|keep|²) – once.
Memory: the old matrix is discarded immediately; only the pruned one remains.
All subsequent operations (update, get_history, project, …) run on the smaller matrix.

In [15]:
rel = HashRelationTensor()

# build a graph with 4 tokens
t = [hash(w) for w in ["A", "B", "C", "D"]]
rel.update(t[0], t[1], 1.0)
rel.update(t[1], t[2], 2.0)
rel.update(t[2], t[3], 3.0)
rel.update(t[0], t[3], 4.0)

print("before prune", rel.R.shape)      # 4×4

pruned = rel.prune([t[0], t[2], t[3]])  # drop B
print("after prune", pruned.shape)      # 3×3
print(rel.h2i)                          # {hash_A:0, hash_C:1, hash_D:2}

# history/future now work only on the pruned set
print(rel.get_history(t[3]))            # {hash_A:4.0, hash_C:3.0}

before prune torch.Size([16, 16])
after prune torch.Size([3, 3])
{6799080192888523577: 0, 7451729274442981443: 1, 4782691667229046252: 2}
{6799080192888523577: 4.0}


## Complexity

- update/get: O(1) average time, one tensor write.
- memory: O(N²) where N = number of distinct hashes (only the actually used N×N block is returned by get_dense()).
- growth: amortised O(1); when capacity doubles we pay one index_copy of the existing N×N block.

## Extensions

1. Symmetric relation: mirror each write R[j,i] = R[i,j].
2. Directed vs undirected: store only upper triangle and use a triu_indices view.
3. Mini-batch updates: vectorise with index_put_ and two integer tensors rows, cols.